# Anillos de polinomios

Existen varias formas de definir un anillo de polinomios en `sage`.

El manual de referencia para este bloc es [Sage Reference Manual: Polynomials](http://doc.sagemath.org/pdf/en/reference/polynomial_rings/polynomial_rings.pdf).

In [3]:
P=PolynomialRing(QQ,['x', 'y'])

In [4]:
P

Multivariate Polynomial Ring in x, y over Rational Field

In [5]:
x^2

x^2

In [6]:
x in P

True

In [7]:
P==QQ['x','y']

True

In [8]:
P2.<x,y>=QQ[]

In [9]:
P2==P

True

## Órdenes monomiales

Por defecto se utiliza el graduado lexicográfico inverso `degrevlex`.

In [10]:
P.term_order()

Degree reverse lexicographic term order

In [11]:
P2.term_order()

Degree reverse lexicographic term order

In [12]:
x<y

False

In [13]:
x>y

True

Tenemos los siguientes órdenes a nuestra disposición:

- `degrevlex` graduado lexicográfico inverso, que es el que viene dado por defecto

- `lex` el lexicográfico

- `invlex` lexicográfico inverso

- `deglex` graduado lexicográfico

- Órdenes por bloques

In [14]:
P=PolynomialRing(QQ,x,5,order='lex')

In [15]:
P

Multivariate Polynomial Ring in x0, x1, x2, x3, x4 over Rational Field

In [16]:
x0<x1

NameError: name 'x0' is not defined

Para poder utilizar de forma interactiva las variables usamos `inject_variables`.

In [17]:
P.inject_variables()

Defining x0, x1, x2, x3, x4


In [18]:
x0<x1

False

In [19]:
x0>x1

True

In [20]:
x0>x1^2

True

In [21]:
P=PolynomialRing(QQ,x,5,order='deglex')

In [22]:
P.inject_variables()

Defining x0, x1, x2, x3, x4


In [23]:
x0>x1^2

False

In [24]:
x0>x1

True

In [25]:
P=PolynomialRing(QQ,['x','y'], order='invlex')

In [26]:
P.inject_variables()

Defining x, y


In [27]:
x<y

True

## Polinomios en una variable

In [28]:
R=Zmod(2)['x']
R.inject_variables()

Defining x


In [29]:
list(R.monics(max_degree=3))

[1,
 x,
 x + 1,
 x^2,
 x^2 + 1,
 x^2 + x,
 x^2 + x + 1,
 x^3,
 x^3 + 1,
 x^3 + x,
 x^3 + x + 1,
 x^3 + x^2,
 x^3 + x^2 + 1,
 x^3 + x^2 + x,
 x^3 + x^2 + x + 1]

In [30]:
R.irreducible_element(3)

x^3 + x + 1

In [31]:
R.irreducible_element(3,algorithm='primitive')

x^3 + x + 1

In [32]:
Q=R.quo(_*R,'a')

In [33]:
Q

Univariate Quotient Polynomial Ring in a over Ring of integers modulo 2 with modulus x^3 + x + 1

In [35]:
a=Q.0

In [36]:
set(a^i for i in range(7))

{1, a, a + 1, a^2, a^2 + 1, a^2 + a, a^2 + a + 1}

Vamos ahora a construir un cuerpo con 256 elementos.

In [37]:
p=R.irreducible_element(8)

In [38]:
Q=R.quo(p*R,'a')

In [39]:
Q

Univariate Quotient Polynomial Ring in a over Ring of integers modulo 2 with modulus x^8 + x^4 + x^3 + x^2 + 1

In [40]:
Q.is_field()

True

In [42]:
a=Q.0
len(set(a^i for i in range(2^8-1)))

255

La multiplicación en `Q` se puede hacer por logaritmos. Guardamos en una lista todas las potencias de `a`, que son todos los elementos de `Q`.

In [43]:
l=[a^i for i in range(2^8-1)]

Vamos a multiplicar `a^2+1` por `a^3+a` (el primero representa al 101 en binario, y el segundo a 10010, es decir, 5 y 18 en decimal, respectivamente). Calculamos su posición en la lista `l`, que coincide con el logaritmo en base `a`, y sumamos las posiciones.

In [53]:
l.index(a^2+1)

50

In [54]:
l.index(a^3+a)

51

In [46]:
l[101]

a^5 + a

In [55]:
(a^3+a)*(a^2+1)

a^5 + a

Si queremos calcular inversos, como $a^{255}=1$, tenemos que si $b=a^x$, entonces $$b^{-1}=1\times a^{-x}=a^{255-x}.$$

In [56]:
1/(a^2+1)

a^7 + a^5 + a^2 + a + 1

In [57]:
l[255-50]

a^7 + a^5 + a^2 + a + 1

Así es como se lleva a cabo la multiplicación y la división en [AES](https://es.wikipedia.org/wiki/Advanced_Encryption_Standard), aunque el polinomio que se usa en ese sistema es $x^8+x^4+x^3+x+1$.

Podemos pasar de un elemento de `Q` a `R` levantándolo.

In [58]:
p=a^8+1

In [59]:
p

a^4 + a^3 + a^2

In [60]:
pl=p.lift()

In [61]:
cf = pl.coefficients(sparse=False)

In [62]:
cf.reverse()

In [63]:
cf

[1, 1, 1, 0, 0]

También podemos pasar de coeficientes a polinomios de forma sencilla.

In [66]:
Q([0,0,1,1,1])

a^4 + a^3 + a^2

Los polinomios primitivos en una variable en $\mathbb Z_2[x]$ también tienen aplicaciones interesantes en la construcción de [LFSR](https://es.wikipedia.org/wiki/LFSR)s, que se usan para cifrar las comunicaciones GSM, [A5/1](https://es.wikipedia.org/wiki/A5/1).

Los LFSR (y las combinaciones de éstos) se usan para crear secuencias de números (pseudo)aleatorios. Estas secuencias las comparten el emisor y el receptor en una conversación. Para cifrar un mensaje basta con hacer posición a posición una suma en $\mathbb Z_2$ del mensaje y la secuencia creada, y enviar el resultado. El receptor repite la operación y recupera el mensaje original: $x+y+y=x$ en $\mathbb Z_ 2$.

## Interpolación

In [67]:
I=4*ZZ

In [70]:
P=QQ["x, y"]

In [73]:
I=(x^2+1, y^4)*P

In [74]:
I.radical()

Ideal (y, x^2 + 1) of Multivariate Polynomial Ring in x, y over Rational Field